In [1]:
import os
import datetime
import polars as pl
import pandas as pd
pl.Config(tbl_rows=100)
from src.najdi_rok import najdi_rok
from src.pocet_stran import pocet_stran
from src.bez_bordelu import bez_bordelu
from src.alt_friendly import alt_friendly
from src.zjisti_vazbu import zjisti_vazbu

In [2]:
df = pl.read_parquet(os.path.join("data/cnb_sloupce","100.parquet"))
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","leader.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","008.parquet")), left_on="001", right_on="001", how="left")
df = df.to_pandas()
df = df[df["leader"].str[6].isin(["a", "t"])]
df = df[~df["leader"].str[7].isin(["b", "i", "s", " "])]
df = df[(df["008"].str[15:17] == "xr") & (df["008"].str[35:38] == "cze")]
df = pl.from_pandas(df)
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","020.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","022.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","245.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","300.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","655.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","700.parquet")), left_on="001", right_on="001", how="left")
df = df.explode("022_a").filter(pl.col("022_a").is_null())
df = df.with_columns(pl.col('008').map_elements(najdi_rok, return_dtype=int).alias('rok'))
df = df.with_columns(pl.col('300_a').map_elements(pocet_stran, return_dtype=int).alias('stran'))
df = df.with_columns(pl.col('245_a').map_elements(bez_bordelu, return_dtype=str))
df = df.explode("020_q").with_columns(pl.col("020_q").map_elements(zjisti_vazbu, return_dtype=str).alias('vazba'))
df = df.explode('245_p').with_columns(pl.col('245_p').map_elements(bez_bordelu, return_dtype=str))
print(len(df))
df = df.filter(pl.col("rok") >= 1800)

df = df.filter((~pl.col("245_h").str.contains("grafika")) | pl.col("245_h").is_null()).unique(subset=["008","100_a","245_a","245_p"], keep="first")
print(len(df))

795736
705921


In [3]:
import requests
from bs4 import BeautifulSoup

## Le Monde's 100 Books of the Century

In [86]:
r = requests.get("https://en.wikipedia.org/wiki/Le_Monde%27s_100_Books_of_the_Century")
soup = BeautifulSoup(r.text)
tabulka_monde = soup.find(class_="sortable")

In [88]:
for radek in tabulka_monde.find_all("tr"):
    try:
        autor = radek.find_all("td")[2].text.strip().split(' ')[-1]
        print(f"{autor}: {radek.find_all("td")[1].text.strip()}")
        print(', '.join(set(df.filter(pl.col('100_a').str.contains(autor)).select(pl.col('245_a')).to_series().to_list())))
        print('***')
    except:
        pass

Camus: The StrangerThe Outsider
Zápisníky I, Pád, Zápisníky II, Exil a království, Mor, První člověk, Nedorozumění, Caligula, Léto, Vyprávění severoamerických indiánů, Romány a povídky, Svět Židů, Cizinec, Mýtus o Sisyfovi, Eseje, Člověk revoltující, Šťastná smrt, Velká výměna, Zápisníky III, Král Roman, Spravedliví
***
Proust: In Search of Lost TimeRemembrance of Things Past
Dnové četby, Myšlenky, Eseje, Swannova láska, Jean Santeuil, Hledání ztraceného času, Radosti a dny, Vzpomínky
***
Kafka: The Trial
Pediatrická chirurgie 1963, Geologie a nauka o ložiskách, Krutá léta, Deníky z cest, Nezvěstný (Amerika), Ivan Kafka 1975-2005, Hup a Hop, Řeka v prstech, Betlemáři, Podmalby na skle ze sbírek Muzea v Bruntále, Josef Jambor, Život naživo, Největší sluha člověka - slon, V hodině velké zkoušky, Úsporná výživa hospodářská a zdravotní reforma výživy, jídelního lístku, vaření a zažívání nejen se zřetelem na nutná obmezení v době války, nýbrž i se zřetelem na hospodářské a zdravotní povznes

Nevyšlo:
- Colette: The Tendrils of the Vine- Duras: The Ravishing of Lol Stein
- Sarraute: Tropisms- Renard: Journal, 1887–1910- Lacan: Écrits
- Cendrars: Moravagine- Simenon: The Strange Case of Peter the Lett
- Genet: Our Lady of the Flowers- Char: Furor and Mystery
- Jacobs: Blake and Mortimer
- Butor: Second Thoughts
- Perse: Amers

Vyšlo, není v datech:
- Buzzati: The Tartar Steppe
- Cohen: Belle du Seigneur
- Sagan: Bonjour Tristesse
- Pratt: The Ballad of the Salty Sea






Jiný název:
- Gracq: The Opposing Shore /  Pobřeží Sy
- Clézio: The Interrogation /  Zápis o katas
   Miller: The Rosy Crucifixiontrofěrt



In [73]:
df.filter(pl.col("100_a").str.contains("Char, René"))

100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,001,leader,008,020_q,020_c,020_a,020_z,022_a,022_y,022_z,022_ind1,022_l,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,300_a,300_b,300_c,300_e,300_f,300_3,655_ind2,655_a,655_7,655_2,655_ind1,655_y,655_z,700_ind1,700_a,700_4,700_d,700_7,700_t,700_q,700_l,700_ind2,700_c,700_b,700_i,700_m,700_k,700_n,700_r,700_p,700_o,700_s,700_j,700_x,700_e,700_f,700_5,700_9,700_g,rok,stran,vazba
str,str,str,list[str],str,str,list[str],str,str,str,str,str,str,list[str],list[str],list[str],str,list[str],list[str],str,str,str,str,str,str,str,list[str],str,str,str,str,list[str],list[str],list[str],list[str],str,str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],i64,i64,str
"""1""","""Char, René,""","""jn20000603097""","[""aut""]","""1907-1988""",null,null,null,null,"""cpk20000960335""",""" nam a22 a 4500""","""000410s1999 xr e 0…","""(v knize neuvedeno ;""",null,"[""80-238-5449-6""]",null,null,null,null,null,null,"""1""","""0""","""Zápisky Hypnovy""","""Výslovný úděl /""","""René Char ; uspořádala, z fran…",null,null,null,null,null,"[""108 s. ;""]",null,"[""21 cm""]",null,null,null,"[""7""]","[""citáty""]","[""fd131832""]","[""czenas""]",null,null,null,"[""1""]","[""Kareninová, Anna,""]","[""trl""]","[""1954-""]","[""jn19981001608""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1999,108,null
"""1""","""Char, René,""","""jn20000603097""","[""aut""]","""1907-1988""",null,null,null,null,"""ck8603226""",""" nam a22 4500""","""860626s1985 xr u0…","""(Brož.) :""","[""Kčs 15,00""]",null,null,null,null,null,null,null,"""1""","""0""","""Společná přítomnost""","""[výbor z básnického díla] /""","""René Char ; z franc. přel. a d…",null,null,null,null,null,"[""304 s. ;""]",null,"[""20 cm""]",null,null,null,"[""7""]","[""výbory""]","[""fd133853""]","[""czenas""]",null,null,null,"[""1""]","[""Kundera, Ludvík,""]","[""aft""]","[""1920-2010""]","[""jk01070892""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1985,304,"""brožovaná"""


In [65]:
df.filter(pl.col("100_a") == "Colette,")

100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,001,leader,008,020_q,020_c,020_a,020_z,022_a,022_y,022_z,022_ind1,022_l,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,300_a,300_b,300_c,300_e,300_f,300_3,655_ind2,655_a,655_7,655_2,655_ind1,655_y,655_z,700_ind1,700_a,700_4,700_d,700_7,700_t,700_q,700_l,700_ind2,700_c,700_b,700_i,700_m,700_k,700_n,700_r,700_p,700_o,700_s,700_j,700_x,700_e,700_f,700_5,700_9,700_g,rok,stran,vazba
str,str,str,list[str],str,str,list[str],str,str,str,str,str,str,list[str],list[str],list[str],str,list[str],list[str],str,str,str,str,str,str,str,list[str],str,str,str,str,list[str],list[str],list[str],list[str],str,str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],i64,i64,str
"""0""","""Colette,""","""jn19981000507""","[""aut""]","""1873-1954""",null,null,null,null,"""bk197300781""",""" nam a22 1 4500""","""970923s1973 xr …",null,null,null,null,null,null,null,null,null,"""1""","""0""","""Kočka""","""Duo /""","""[autorka] Colette ; z francouz…",null,null,null,null,null,"[""264, [3] s. ;""]",null,"[""8°""]",null,null,null,null,null,null,null,null,null,null,"[""1"", ""1""]","[""Ondrušková, Alena,"", ""Musilová, Eva,""]","[""aui"", ""trl""]","[""1923-"", ""1925-2009""]","[""jk01091132"", ""jk01082537""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1973,264,null
"""0""","""Colette,""","""jn19981000507""","[""aut""]","""1873-1954""",null,null,null,null,"""bk193500448""",""" nam a22 1 4500""","""990405s1935 xr a 0…","""(Váz.)""",null,null,null,null,null,null,null,null,"""1""","""0""","""Klaudina ve škole""",null,"""Colette ; [přeložila Jar. Vobr…",null,null,null,null,null,"[""286 - [II] s. :""]","[""il. ;""]","[""8°""]",null,null,null,null,null,null,null,null,null,null,"[""1"", ""1""]","[""Vobrubová-Koutecká, Jaroslava,"", ""Mirande, Henri,""]","[""trl"", ""ill""]","[""1891-1969"", ""1877-1955""]","[""jk01150197"", ""xx0133044""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1935,286,"""pevná"""
"""0""","""Colette,""","""jn19981000507""","[""aut""]","""1873-1954""",null,null,null,null,"""bk193500446""",""" nam a22 1 4500""","""990405s1935 xr a 0…","""(Váz.)""",null,null,null,null,null,null,null,null,"""1""","""0""","""Klaudina odchází""","""[deník Annie Samzunové] /""","""Colette ; [přeložil Břetislav …",null,null,null,null,null,"[""220 s. :""]","[""il. ;""]","[""8°""]",null,null,null,null,null,null,null,null,null,null,"[""1""]","[""Koudelka, Břetislav""]","[""trl""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1935,220,"""pevná"""
"""0""","""Colette,""","""jn19981000507""","[""aut""]","""1873-1954""",null,null,null,null,"""cpk20081798991""",""" nam a22 i 4500""","""080403s1933 xr e 0…","""(Vázáno)""",null,null,null,null,null,null,null,null,"""1""","""0""","""Náhražka lásky""","""L'ersatz d'amour : román /""","""Willy - Menalkas ; přeložil Je…",null,null,null,null,null,"[""193 stran ;""]",null,"[""19 cm""]",null,null,null,"[""7""]","[""francouzské romány""]","[""fd132289""]","[""czenas""]",null,null,null,"[""0"", ""1""]","[""Ménalkas,"", ""Lipanský, Jetřich,""]","[""aut"", ""trl""]","[""1883-1964"", ""1900-1971""]","[""jo2014836450"", ""jk01072208""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1933,193,"""pevná"""
"""0""","""Colette,""","""jn19981000507""","[""aut""]","""1873-1954""",null,null,null,null,"""bk193100446""",""" nam a22 1 4500""","""990719s1931 xr …",null,null,null,null,null,null,null,null,null,"""1""","""0""","""Zlatíčko""",null,"""Colette ; [

## Bokklubben World Library

In [90]:
r = requests.get("https://en.wikipedia.org/wiki/Bokklubben_World_Library")
soup = BeautifulSoup(r.text)
tabulka_bokklubben = soup.find(class_="sortable")

In [93]:
len(tabulka_bokklubben.find_all("tr"))

101

In [92]:
for radek in tabulka_bokklubben.find_all("tr"):
    try:
        autor = radek.find_all("td")[1].text.strip().split(' ')[-1]
        print(f"{autor}: {radek.find_all("td")[0].text.strip()}")
        print(', '.join(set(df.filter(pl.col('100_a').str.contains(autor)).select(pl.col('245_a')).to_series().to_list())))
        print('***')
    except:
        pass

Achebe: Things Fall Apart
Muž z lidu, Svět se rozpadá, Už nikdy klid
***
Andersen: Fairy Tales
Jane Fondová, Andersenovy Vybrané báchorky, Ten muž má něco do sebe, Křišťálové vidiny, Cínový vojáček, Pohádky a povídky, Nauč mne věřit ve štěstí, Tajný život v Bílém domě, Dvanáct poštou, Najděte své ideální zdraví!, Dospělost je dar!, Hans Christian Andersen: Nejkrásnější pohádky, Čarovné zjevy, V železné době, Císařovy nové šaty, Grimmové, Ošklivé kačátko, O dívce, která šlápla na chléb, Jackie po Jackovi, Dospělost je mýtus, Andersenovy pohádky, Mimoňka, Rudý Morten, Obrázková kniha bez obrázků, Osudná závěť, Andersenovy báchorky, Vzpomínky, Zámek u jezera, Milovat se zakazuje, Ilustrované pohádky, Má velká doba byla malá, Ditta, dcera člověka, Ošklivé káčátko, Ďábel loterie, Pohádka mého života, Vybrané Andersenovy pohádky, Štěstí, Kočky útočí, Dvě baronky, Improvisator, Křesadlo, Pelle dobyvatel, Ztracená generace, Výhra, Jack a Jackie Kennedyovi, Zpráva o cestě do Saského Švýcarska, 

In [44]:
df.filter(pl.col("245_a").str.contains("Mahá")).select(pl.col("245_a")).to_series().to_list()

['Světlo Asie, čili, Veliké odříkání (Mahábhiniškramana), to jest život a učení Gótamy, prince indického a zakladatele buddhismu (jak vyprávěno ve verších indickým buddhistou)',
 'S Ramanou Mahárišim den za dnem',
 '15 dní s Mahátma Gándhím',
 'Mahátma Gándhí',
 'Mahámudra',
 'Mahájánské texty',
 'Mahárádžův miláček',
 'Bhagavan Šrí Ramana Maháriši, mudrc z Arunáčaly',
 'Mahábhárata',
 'Mahárišiho evangelium',
 'Mahábhárata',
 'Mahárádžův miláček',
 'Světlo na učení Bhagavána Šrí Ramana Maháršiho',
 'Tak pravil Ramana Maháriši',
 'Bhagavan Šrí Ramana Maháriši, mudrc z Arunáčaly',
 'Mahájánový buddhismus',
 'Rozhovory s Ramanou Mahárišim',
 'Mahátmá Gándhí',
 'Ježíš Kristus a Ramana Maháriši',
 'Život a učení Ramany Mahárišiho',
 'Mahátma Gándhí',
 'Mahátma Gándhí',
 'Mahárádžova nevěsta',
 'Mahátmá Gándhí',
 'Mahátma Gándhí',
 'Mahábhárata I',
 'Mahámudrá',
 'Zlatý avatár Šrí Čaitanja Maháprabhu',
 'Mahábhárata, aneb, Velký boj',
 'Zamyšlení nad výroky Ramana Mahárišiho všeobecně a v k

In [32]:
df.filter(pl.col("245_a").str.contains("Sága")).select(pl.col("245_a")).to_series().to_list()

['Sága rodu Forsytů',
 'Sága rodu Forsytů',
 'Sága rodu Porsche',
 'Sága rodu',
 'Sága našeho rodu',
 'Sága zapomenutého válečníka',
 'Sága od Bodamského jezera',
 'Sága rodu Forsytů',
 'Sága o Volsunzích',
 'Sága Lartena Hroozleyho',
 'Sága Sirionů',
 'Sága o životě a smrti Jana Bati a jeho bratra Tomáše',
 'Sága',
 'Sága rodu Reissových',
 'Sága o Salavinovi',
 'Sága Sirionů',
 'Sága rodu Floriů',
 'Sága rodu Benešů',
 'Sága o životě a smrti Jana Bati a jeho bratra Tomáše',
 'Sága Sirionů',
 'Sága Lartena Hroozleyho',
 'Sága Sirionů',
 'Sága o Hákonovi',
 'Sága rodiny Škodů',
 'Sága rodu Chlebakupova',
 'Sága temných templářů',
 'Sága pana Janíčka',
 'Sága o Lundirovi - synu zimy, hvězdopravci a staviteli',
 'Sága rodu Budilova',
 'Sága rodu Forsytů',
 'Sága rodu křídloprachů',
 'Sága o cestě na Island',
 'Sága od Bodamského jezera',
 'Sága z Valhally',
 'Sága o Halldorovi z Mortaluny',
 'Sága o Narviku',
 'Sága',
 'Sága rodu Rotwengelů',
 'Sága Sirionů',
 'Sága temných templářů',
 '

Nevyšlo:
- Celan: Poems
- Rosa: The Devil to Pay in the Backlands
- Lessing: The Golden Notebook
- Salih: Season of Migration to the North

spíš vyšlo:
- Rumi: Masnavi
- fróði: Njál's Saga

In [82]:
df.filter(pl.col("100_a").str.contains("Celan, Paul"))

100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,001,leader,008,020_q,020_c,020_a,020_z,022_a,022_y,022_z,022_ind1,022_l,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,300_a,300_b,300_c,300_e,300_f,300_3,655_ind2,655_a,655_7,655_2,655_ind1,655_y,655_z,700_ind1,700_a,700_4,700_d,700_7,700_t,700_q,700_l,700_ind2,700_c,700_b,700_i,700_m,700_k,700_n,700_r,700_p,700_o,700_s,700_j,700_x,700_e,700_f,700_5,700_9,700_g,rok,stran,vazba
str,str,str,list[str],str,str,list[str],str,str,str,str,str,str,list[str],list[str],list[str],str,list[str],list[str],str,str,str,str,str,str,str,list[str],str,str,str,str,list[str],list[str],list[str],list[str],str,str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],i64,i64,str
"""1""","""Celan, Paul,""","""jn20030827047""","[""aut""]","""1920-1970""",null,null,null,null,"""gk48090719""",""" nam a22 4500""","""870320s1986 xr u0…","""(Váz.) :""","[""18 Kčs""]",null,null,null,null,null,null,null,"""1""","""0""","""Sněžný part""","""[Výbor básní] /""","""Paul Celan ; Vybral, z něm. př…",null,null,null,null,null,"[""230 s. ;""]",null,"[""19 cm""]",null,null,null,"[""7""]","[""výbory""]","[""fd133853""]","[""czenas""]",null,null,null,"[""1""]","[""Kundera, Ludvík,""]","[""aui""]","[""1920-2010""]","[""jk01070892""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1986,230,"""pevná"""
"""1""","""Celan, Paul,""","""jn20030827047""","[""aut""]","""1920-1970""",null,null,null,null,"""nkc20122433087""",""" nam a22 a 4500""","""130514s2012 xr a g 0…","""(brož.)""",null,"[""978-80-260-3653-1""]",null,null,null,null,null,null,"""1""","""0""","""Báseň - domov v bezdomoví Paul…",null,"""výbor z básnického díla, překl…",null,null,null,null,null,"[""115 s. :""]","[""il. ;""]","[""21 cm""]",null,null,null,"[""7"", ""7"", … ""9""]","[""německy psaná poezie"", ""výbory"", … ""jubilee books""]","[""fd132931"", ""fd133853"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,"[""1"", ""1""]","[""Syřišťová, Eva,"", ""Siostrzonek, Jiří,""]","[""aft"", ""ill""]","[""1928-2016"", ""1954-""]","[""jk01122288"", ""jo20000071800""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2012,115,"""brožovaná"""
"""1""","""Celan, Paul,""","""jn20030827047""","[""aut""]","""1920-1970""",null,null,null,null,"""nkc20152722707""",""" cam a22 i 4500""","""150709s2015 xr a e 0…","""(brožováno)""",null,"[""978-80-87545-37-9""]",null,null,null,null,null,null,"""1""","""0""","""Černé vločky""",null,"""Paul Celan ; vybral, uspořádal…",null,null,null,null,null,"[""206 stran :""]","[""ilustrace ;""]","[""16 cm""]",null,null,null,"[""7"", ""7"", … ""9""]","[""německy psaná poezie"", ""výbory"", … ""selected works""]","[""fd132931"", ""fd133853"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,"[""1""]","[""Malý, Radek,""]","[""trl""]","[""1977-""]","[""xx0000314""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2015,206,"""brožovaná"""
"""1""","""Celan, Paul,""","""jn20030827047""","[""aut""]","""1920-1970""",null,null,null,null,"""nkc20243628530""",""" nam a22 i 4500""","""240830s2024 xr g 0…","""(vázáno) :""","[""Kč 277,00""]","[""978-80-7690-101-8""]",null,null,null,null,null,null,"""1""","""0""","""Všechno je jinak""",null,"""Paul Celan ; přeložil Ludvík K…",null,null,null,null,null,"[""275 stran ;""]",null,"[""19 cm""]",null,null,null,"[""7"", ""7"", … ""9""]","[""německy psaná poezie"", ""výbory"", … ""selected works""]","[""fd132931"", ""fd133853"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,"[""1""]","[""Kundera, Ludv

## The New York Times' 100 Best Books of the 21st Century

In [97]:
r = requests.get("https://en.wikipedia.org/wiki/The_New_York_Times%27_100_Best_Books_of_the_21st_Century")
soup = BeautifulSoup(r.text)
tabulka_21 = soup.find(class_="sortable")

In [139]:
lide = []
for radek in tabulka_21.find_all("tr"):
    if "20" in radek.text:
        autorstvo = radek.find_all("td")[2].text.split(' ')[-1]
        autorstvo_krestni = radek.find_all("td")[2].text.split(' ')[0]
        lide.append(f"{autorstvo_krestni}: {autorstvo}")
len(set(lide))

87

In [131]:
for radek in tabulka_21.find_all("tr"):
    if "20" in radek.text:
        titul = radek.find_all("td")[1].text
        autorstvo = radek.find_all("td")[2].text.split(' ')[-1]
        autorstvo_krestni = radek.find_all("td")[2].text.split(' ')[0]
        print(f"{radek.find_all('td')[2].text}: {titul}")
        print(', '.join(set(df.filter(pl.col('100_a').str.contains(autorstvo)).filter(pl.col('100_a').str.contains(autorstvo_krestni)).select(pl.col('245_a')).to_series().to_list())))
        print("***")

Elena Ferrante: My Brilliant Friend
Na okraji, Prolhaný život dospělých, Temná dcera, Pláž v noci, Příležitostné nápady, Dny opuštění, Geniální přítelkyně, Tíživá láska
***
Isabel Wilkerson: The Warmth of Other Suns

***
Hilary Mantel: Wolf Hall
Za temnotou, Zrcadlo a světlo, Předveďte mrtvé, Wolf Hall, Zavraždění Margaret Thatcherové a jiné povídky
***
Edward P. Jones: The Known World
Hotel Babylon, aneb, Nechcete raději zůstat doma?, Letiště Babylon, aneb, Nechcete raději jet vlakem?, Access 2000, Módní salon Babylon, aneb, Nechcete raději nakupovat konfekci?
***
Jonathan Franzen: The Corrections
Purity, Svoboda, Rozhřešení
***
Roberto Bolaño: 2666
Nacistická literatura v Americe, Amulet, 2666, Divocí detektivové, Chilské nokturno, Třetí říše, Vzdálená hvězda
***
Colson Whitehead: The Underground Railroad
Podzemní železnice, Zóna jedna
***
W.G. Sebald: Austerlitz

***
Kazuo Ishiguro: Never Let Me Go
Vybledlá krajina s kopci, Soumrak dne, Neopouštěj mě, Klára a Slunce, Malíř pomíjivéh

Nevyšlo:
- Isabel Wilkerson: The Warmth of Other Suns- Edward P. Jones: The Known World
- Patrick Radden Keefe: Say Nothing
- Richard Powers: The Overstory
- vůbec: - Matthew Desmond: Evicted
- vůbec: Claudia Rankine: Citizen
- Ta-Nehisi Coates: Between the World and Me
- Marlon James: A Brief History of Seven Killings
- N. K. Jemisin: The Fifth Season
- vůběc: Maggie Nelson: The Argonauts
- vůbec: Hernan Diaz: Trust
- Rachel Kushner: The Flamethrowers
- vůbec: Barbara Ehrenreich: Nickel and Dimed
- Hua Hsu: Stay True
- vůbec: Kiese Laymon: Heavy
- Barbara Kingsolver: Demon Copperhead
- Ben Lerner: 10:04- vůbec: Mary Gaitskill: Veronica- vůbec: Rebecca Makkai: The Great Believers
- vůbec: Andrew Solomon: Far From the Tree- vůbec: *
Sigrid Nunez: The Frie- vůbec: 
***
Michelle Alexander: The New Jim- Edward P. Jones: All Aunt Hagar's Children
- (asi vůbec) Robert Caro: The Passage of Power
- John Jeremiah Sullivan: Pulphead
- George Saunders: Pastoralia
- vůbec: David W. Blight: Frederick Douglass- vůbec: *
Torrey Peters: Detransition, Ba- vůbec: 
***
Lydia Davis: The Collected Stories of Lydia
- Hisham Matar: The Return
- Saidiya Hartman: Wayward Lives, Beautiful Experiments
- Jesmyn Ward: Men We Reaped Davis
 Crow



Težko ří
- Denis Johnson: Train Dreamsct:
- Percival Everett: Erasu- Lawrence Wright: The Looming Towerte
- Ian McEwan: Atonementd


In [133]:
df.filter(pl.col("100_a").str.contains("Rankine, Clau"))

100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,001,leader,008,020_q,020_c,020_a,020_z,022_a,022_y,022_z,022_ind1,022_l,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,300_a,300_b,300_c,300_e,300_f,300_3,655_ind2,655_a,655_7,655_2,655_ind1,655_y,655_z,700_ind1,700_a,700_4,700_d,700_7,700_t,700_q,700_l,700_ind2,700_c,700_b,700_i,700_m,700_k,700_n,700_r,700_p,700_o,700_s,700_j,700_x,700_e,700_f,700_5,700_9,700_g,rok,stran,vazba
str,str,str,list[str],str,str,list[str],str,str,str,str,str,str,list[str],list[str],list[str],str,list[str],list[str],str,str,str,str,str,str,str,list[str],str,str,str,str,list[str],list[str],list[str],list[str],str,str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],i64,i64,str
